In [1]:
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder.appName("test").getOrCreate()

In [3]:
data = [
    [100, "Mobile", 5000, 10],
    [100, "Mobile", 7000, 7],
    [200, "Laptop", 20000, 4],
    [200, "Laptop", 25000, 8],
    [200, "Laptop", 22000, 12],
]
df = spark.createDataFrame(data, ["product_id", "name", "price", "discount_pct"])
df.show()

+----------+------+-----+------------+
|product_id|  name|price|discount_pct|
+----------+------+-----+------------+
|       100|Mobile| 5000|          10|
|       100|Mobile| 7000|           7|
|       200|Laptop|20000|           4|
|       200|Laptop|25000|           8|
|       200|Laptop|22000|          12|
+----------+------+-----+------------+



In [4]:
price_window_spec = Window.partitionBy("product_id").orderBy(desc("price"))
df = df.withColumn("price_max", first("price").over(price_window_spec))

discount_window_spec = Window.partitionBy("product_id").orderBy(desc("discount_pct"))
df = df.withColumn("discount_pct_max", first("discount_pct").over(discount_window_spec))

df.show()

+----------+------+-----+------------+---------+----------------+
|product_id|  name|price|discount_pct|price_max|discount_pct_max|
+----------+------+-----+------------+---------+----------------+
|       100|Mobile| 5000|          10|     7000|              10|
|       100|Mobile| 7000|           7|     7000|              10|
|       200|Laptop|22000|          12|    25000|              12|
|       200|Laptop|25000|           8|    25000|              12|
|       200|Laptop|20000|           4|    25000|              12|
+----------+------+-----+------------+---------+----------------+



In [8]:
df.select(
    col("product_id"),
    col("name"),
    col("price_max").alias("price"),
    col("discount_pct_max").alias("discount_pct"),
).distinct().show()

+----------+------+-----+------------+
|product_id|  name|price|discount_pct|
+----------+------+-----+------------+
|       100|Mobile| 7000|          10|
|       200|Laptop|25000|          12|
+----------+------+-----+------------+

